# ft_printf

## Makefile

In [71]:
%%file ../ft_printf/Makefile
# **************************************************************************** #
#                                                                              #
#                                                         :::      ::::::::    #
#    Makefile                                           :+:      :+:    :+:    #
#                                                     +:+ +:+         +:+      #
#    By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+         #
#                                                 +#+#+#+#+#+   +#+            #
#    Created: 2022/12/29 22:12:48 by dpentlan          #+#    #+#              #
#    Updated: 2022/12/29 22:12:50 by dpentlan         ###   ########.fr        #
#                                                                              #
# **************************************************************************** #

NAME = libftprintf.a

SRCS = ft_printf.c \
    ft_printf_utils_1.c \
    ft_printf_utils_2.c

OBJS = ${SRCS:.c=.o}

INCS = ft_printf.h

CFLAGS = -Wall -Wextra -Werror

CC = cc

SLIB = ar rcs

all: $(NAME)

$(NAME): $(OBJS) $(INCS)
	$(SLIB) $(NAME) -o $(OBJS)

%.o: %.c
	$(CC) $(CFLAGS) -c $< -o $(<:.c=.o)

clean:
	rm -f $(OBJS)

fclean:	clean
	rm -f $(NAME)

re:	fclean all


Overwriting ../ft_printf/Makefile


## ft_printf.h

In [72]:
%%file ../ft_printf/ft_printf.h
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   ft_printf.h                                      :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/30 18:32:59 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/30 18:33:01 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#ifndef FT_PRINTF_H
# define FT_PRINTF_H
# include <stdarg.h>
# include <stddef.h>
# include <unistd.h>

int ft_printf(const char *, ...);
void	ft_putchar_fd(char c, int fd);
void	ft_putstr_fd(char *s, int fd);
void	ft_putnbr_fd(int n, int fd);

#endif

Overwriting ../ft_printf/ft_printf.h


## ft_printf.c

In [73]:
%%file ../ft_printf/ft_printf.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   ft_printf.c                                        :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/31 14:22:48 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/31 14:22:49 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "ft_printf.h"

/*  *** format_switch (format switch) ***
 *
 *  Acts as a switch statement that determines which function to call
 *  Based on the format specifier used in ft_printf.
 *	Takes the character after a % from input string s in function ft_printf
 *	Takes va_list pointer 'parg'.
 *	Runs va_arg macro to get the next item in va_list. At this point we know
 *	the type thanks to the switch statement.
 *	Returns integer 1 on success.
 */

static int	format_switch(char c, va_list parg)
{
	if (c == 'c')
		ft_putchar_fd(va_arg(parg, int), 1);
	else if (c == 's')
		ft_putstr_fd(va_arg(parg, char *), 1);
	//else if (c == 'p')
	else if (c == 'd')
		ft_putnbr_fd(va_arg(parg, int), 1);
	else if (c == 'i')
		ft_putnbr_fd(va_arg(parg, int), 1);
	//else if (c == 'u')
	//else if (c == 'x')
	//else if (c == 'X')
	//else if (c == '%')
	return (1);
}

/*  *** ft_printf (42 print format) ***
 *
 *  Takes a string followed by a variable number of arguments corresponding
 *  to format specifiers in the string.
 *	Uses a loop to iterate over string. When a % is found, activates a switch
 *	statement function and passes pointer to va_list for further execution.
 *	Returns integer number of character processed from string 's'.
 */

int ft_printf(const char *s, ...)
{
	size_t	i;
	va_list	parg;

	i = 0;
	va_start(parg, s);
	while (s[i])
	{
		if (s[i] == '%')
		{
			format_switch(s[i + 1], parg);
			i++;
		}
		else
		{
			write(1, &s[i], 1);
		}
		i++;
	}
	va_end(parg);
	return (i);
}

Overwriting ../ft_printf/ft_printf.c


## ft_printf_utils_1.c

In [74]:
%%file ../ft_printf/ft_printf_utils_1.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   ft_printf_utils_1.c                                :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/31 14:41:16 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/31 14:41:19 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "ft_printf.h"

/*  *** ft_putchar_fd (42 put character file descriptor) ***
 *	*** from libft ***
 *
 *  Outputs the character 'c' to file descriptor.
 *  Returns nothing.
 */

void	ft_putchar_fd(char c, int fd)
{
	write(fd, &c, 1);
}

/*  *** ft_strlen (42 string length) ***
 *	*** from libft ***
 *
 *  Takes a character pointer.
 *  Returns size_t number of 8 bit memory spaces between input pointer and
 *  the first null terminator.
 */

size_t	ft_strlen(const char *str)
{
	size_t	i;

	i = 0;
	while (str && str[i])
		i++;
	return (i);
}

/*  *** ft_putstr_fd (42 put string file descriptor) ***
 *	*** from libft ***
 *
 *  Outputs the string 's' to the given file descriptor.
 *  Returns nothing.
 */

void	ft_putstr_fd(char *s, int fd)
{
	if (s == 0)
		return ;
	write(fd, s, ft_strlen(s));
}

/*  *** ft_putnbr_fd (42 put number file descriptor) ***
 *
 *  Outputs the integer 'n' to the given file descriptor.
 *  Returns nothing.
 */

void	ft_putnbr_fd(int n, int fd)
{
	long	nbl;
	char	c;

	nbl = (long)n;
	if (nbl < 0)
		write(fd, "-", 1);
	if (nbl / 10 != 0)
	{
		if (nbl < 0)
			ft_putnbr_fd((nbl / 10) * -1, fd);
		else
			ft_putnbr_fd(nbl / 10, fd);
	}
	if (nbl < 0)
		c = (((nbl % 10) * -1) + '0');
	else
		c = (nbl % 10 + '0');
	write(fd, &c, 1);
	return ;
}

Overwriting ../ft_printf/ft_printf_utils_1.c


## ft_printf_utils_2.c

In [75]:
%%file ../ft_printf/ft_printf_utils_2.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   ft_printf_utils_2.c                                :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/31 14:41:27 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/31 14:41:28 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */




Overwriting ../ft_printf/ft_printf_utils_2.c


# Testing

In [76]:
%%file test_ft_printf.c
#include "../ft_printf/ft_printf.h"

int main (void)
{
	ft_printf("Testing character: %c\n", 'W');
	ft_printf("Testing string: %s\n", "Here is a test string.");
	ft_printf("Testing decimal: %d\n", 12345);
	ft_printf("Testing integer: %i\n", 67890);

	return (0);
}

Overwriting test_ft_printf.c


In [79]:
%%bash
gcc test_ft_printf.c -g -L../ft_printf -lftprintf
./a.out

Testing character: W
Testing string: Here is a test string.
Testing decimal: 
Testing integer: 


# Scratch

In [78]:
%%file glibc_printf.c

/* Copyright (C) 1991-2022 Free Software Foundation, Inc.
   This file is part of the GNU C Library.
   The GNU C Library is free software; you can redistribute it and/or
   modify it under the terms of the GNU Lesser General Public
   License as published by the Free Software Foundation; either
   version 2.1 of the License, or (at your option) any later version.
   The GNU C Library is distributed in the hope that it will be useful,
   but WITHOUT ANY WARRANTY; without even the implied warranty of
   MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
   Lesser General Public License for more details.
   You should have received a copy of the GNU Lesser General Public
   License along with the GNU C Library; if not, see
   <https://www.gnu.org/licenses/>.  */
#include <libioP.h>
#include <stdarg.h>
#include <stdio.h>
#undef printf
/* Write formatted output to stdout from the format string FORMAT.  */
/* VARARGS1 */
int
__printf (const char *format, ...)
{
  va_list arg;
  int done;
  va_start (arg, format);
  done = __vfprintf_internal (stdout, format, arg, 0);
  va_end (arg);
  return done;
}
#undef _IO_printf
ldbl_strong_alias (__printf, printf);
ldbl_strong_alias (__printf, _IO_printf);

Overwriting glibc_printf.c
